In [32]:
# setup for jupyter and django
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'aimharder.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [33]:
# remaining libraries 

from datetime import datetime, timedelta
from requests import Session
from bookings.models import Booking


In [ ]:
# aimharder class

class AimHarderSession:

    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.session = Session()
        self.login()
        self.last_response = None
        self.class_list = None
        self.BOX_ID = 8244
        self.BOX_NAME = 'crossfitgrau'
        self.LOGIN_ENDPOINT = 'https://aimharder.com/login'
        self.CLASS_API_ENDPOINT = f'https://{self.BOX_NAME}.aimharder.com/api/bookings'
        self.BOOKINGS_API_ENDPOINT = f'https://{self.BOX_NAME}.aimharder.com/api/book'

    def login(self):
        data = {
            'mail': self.username,
            'pw': self.password,
            'login': 'Log in'
        }
        response = self.session.post(
            self.LOGIN_ENDPOINT,
            data = data
        )
        self.last_response = response

    def get_classes(self, date):
        class_list = self.session.get(
            self.CLASS_API_ENDPOINT,
            params = {
                'day': date,
                'family_id': '',
                'box': self.BOX_ID,
            }
        )
        self.class_list = class_list.json()

    def book_class(self, class_id):
        data = {
            'id': class_id,
            'box': self.BOX_ID,
            'family_id': '',
            'insist': 0,
        }
        response = self.session.post(
            self.BOOKINGS_API_ENDPOINT,
            data = data
        )
        return response

In [50]:
# log in function

# def login(username, password):
#     data = {
#         'mail': username,
#         'pw': password,
#         'login': 'Log in'
#     }
#     session = Session()
#     response = session.post(
#         LOGIN_ENDPOINT,
#         data = data
#     )
#     return [session, response]


In [51]:
# get classes 

# def get_classes(session, date):
#     class_list = session.get(
#         CLASS_API_ENDPOINT,
#         params = {
#             'day': date,
#             'family_id': '',
#             'box': BOX_ID,
#         }
#     )
#     return class_list.json()


In [52]:
# book the class

# def book_class(session, class_id):
#     data = {
#         'id': class_id,
#         'box': BOX_ID,
#         'family_id': '',
#         'insist': 0,
#     }
#     response = session.post(
#         BOOKINGS_API_ENDPOINT,
#         data = data
#     )
#     return response


In [53]:
# all bookings for tomorrow from db

tomorrow = datetime.now() + timedelta(days=1)
bookings = Booking.objects.filter(date=tomorrow)


In [58]:
for booking in bookings:

    # log in
    aimharder = AimHarderSession(booking.user.email, booking.user.password)

    # if login successful
    if aimharder.last_response.status_code == 200:
        
        # get classes
        aimharder.get_classes(tomorrow.strftime("%Y%m%d"))

        # if there are classes
        if aimharder.class_list['bookings']:

            # find the class
            workout = [
                lesson for lesson in aimharder.class_list['bookings'] 
                if lesson['timeid'] == f'{booking.time.strftime("%H%M")}_60' 
                    and lesson['className'] == booking.type
            ][0]

            # book the class
            aimharder.book_class(workout['id'])


In [60]:

response.content

b'{"bookState":-2}'

'7dqzzp%921tfq*gpvbrhonh#mqfy0#k1^a*vro5c=ahm7^q)6@'